In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/output_data/merged_data.csv')

In [ ]:
%%time

# Feature Engineering params
categorical_columns = ['item_id', 'item_category_id']

# Remove shop_id and sum 
df_rs = (
    df_merged[['date', 'item_id', 'item_category_id', 'item_cnt_day']]
    .groupby(['item_id', 'item_category_id', 'date'])
    .sum()
    .fillna(0)
    .reset_index()
)

# Convert required columns to categorical
for col in categorical_columns:
    df_rs[col] = df_rs[col].astype('str').astype('category')

# Unique monthly dates
dates = pd.date_range(
    start=df_rs['date'].min(), 
    end=df_rs['date'].max(),
    freq='M'
)

# Infilling missing dates function
def reindex_by_date(df):
    return df.reindex(dates).fillna(0)

# Resample to Monthly

df_rs = (
    df_rs.set_index('date')
    .groupby(['item_id', 'item_category_id'])
    .resample('M')
    .sum()
    .reset_index()
    .set_index('date')
    .groupby(['item_id', 'item_category_id'])
    .apply(reindex_by_date)
    .reset_index()
    .rename({
        'item_cnt_day': 'item_cnt_month',
        'level_2': 'date'
    }, axis=1)
)